In [20]:
from utils.extract_pdf_contents import process_pdfs
from utils.clean_texts import clean_EU_legal_text, clean_US_legal_text
from utils.tokenize_TFIDF import load_text, compute_tfidf, save_tfidf_tokens
from utils.tokenize_BoW import custom_tokenizer
from sklearn.feature_extraction.text import CountVectorizer
import spacy
import nltk

## Research paper NLP Pipeline

#### 1. Extract the text from the two pdf documents

In [2]:
# Define the input and output directories
input_directory = "data/raw"  # Directory containing the PDF files
output_directory = "data/extracted_text"  # Directory to save the extracted text files


# Call the function to process the PDFs
process_pdfs(input_directory, output_directory)

Extracted text saved to: data/extracted_text/EU_AI_Act_English.txt
Extracted text saved to: data/extracted_text/USA_AI_Executive_Order_English.txt


#### 2. Cleanup the extracted text

In [14]:
# First the EU Act

# Read the file content and pass it to clean_legal_text
with open("data/extracted_text/EU_AI_Act_English.txt", "r", encoding="utf-8") as file:
    text = file.read()

clean_EU_text = clean_EU_legal_text(text)

# Save cleaned text to /data/cleaned_text directory
with open("data/cleaned_text/EU_AI_Act_English_Cleaned.txt", "w", encoding="utf-8") as file:
    file.write(clean_EU_text)


# Second, the USA AI Executive Order

# Read the file content and pass it to clean_legal_text
with open("data/extracted_text/USA_AI_Executive_Order_English.txt", "r", encoding="utf-8") as file:
    text = file.read()
    clean_US_text = clean_US_legal_text(text)

# Save cleaned text to /data/cleaned_text directory
with open("data/cleaned_text/USA_AI_Executive_Order_English_Cleaned.txt", "w", encoding="utf-8") as file:
    file.write(clean_US_text)

#### 3. Text representation via BoW

In [21]:
# Paths to the cleaned text files
text_file_1 = "data/cleaned_text/EU_AI_Act_English_Cleaned.txt"
text_file_2 = "data/cleaned_text/USA_AI_Executive_Order_English_Cleaned.txt"

# Load the cleaned texts
text1 = load_text(text_file_1)
text2 = load_text(text_file_2)

spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm", disable=["tok2vec", "tagger", "parser", "ner",
                                                "lemmatizer", "attibute_ruler"])
nltk.download("stopwords")
stop_words_en = nltk.corpus.stopwords.words("english")


# Vectorizer Configuration for BoW (required for LDA)
tokenizer_bow = CountVectorizer(analyzer="word",
                                       tokenizer=custom_tokenizer,
                                       lowercase=True,
                                       stop_words=stop_words_en,
                                       max_df= 0.80, # Ignore terms that appear in more than 80% of the documents
                                       min_df= 0.01) # Ignore terms that appear in less than 1% of the documents
reddit_bow = tokenizer_bow.fit_transform(text1)
print(f"BoW Matrix Dimensions: {reddit_bow.shape}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━ 9.4/12.8 MB 18.3 kB/s eta 0:03:05


  Resuming download https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (9.4 MB/12.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 37.2 kB/s  0:02:06m0:00:0700:18
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nicolasreichardt/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


ValueError: Iterable over raw text documents expected, string object received.

#### 4. Text representation via TF-IDF

In [4]:
if text1 and text2:
    # Compute TF-IDF
    feature_names, tfidf_matrix = compute_tfidf([text1, text2])
    
    # Save TF-IDF tokens
    output_directory = "data/tfidf_tokens"
    save_tfidf_tokens(output_directory, feature_names, tfidf_matrix)

TF-IDF tokens saved to: data/tfidf_tokens/tfidf_doc_1.txt
TF-IDF tokens saved to: data/tfidf_tokens/tfidf_doc_2.txt


#### 4.2. Text representation through Word2Vec emdeddings

#### 4.3. Text representation through OpenAI contextualised embeddings